In [1]:
import requests
import time
import csv
from bs4 import BeautifulSoup
from random import uniform
from tqdm import tqdm
from fake_useragent import UserAgent

BASE_URL = "https://books.toscrape.com/catalogue/page-{}.html"
TOTAL_PAGES = 50
OUTPUT_CSV = "books.csv"

# Функция для извлечения рейтинга
def get_rating(tag):
    if not tag:
        return None
    classes = tag.get("class", [])
    ratings = ["One", "Two", "Three", "Four", "Five"]
    for r in ratings:
        if r in classes:
            return r
    return None

# Создаём CSV и записываем заголовки
with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["title", "price", "rating", "availability", "book_url"])

# Генератор User-Agent
ua = UserAgent()

# Основной цикл
for page in tqdm(range(1, TOTAL_PAGES + 1), desc="Сбор книг"):
    url = BASE_URL.format(page)

    # Формируем заголовки со случайным User-Agent
    headers = {
        "User-Agent": ua.random,
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Referer": "https://www.google.com/"
    }

    try:
        # Запрос с таймаутом
        resp = requests.get(url, headers=headers, timeout=10)
        resp.raise_for_status()                     # выбросит исключение при 4xx/5xx
    except requests.exceptions.RequestException as e:
        print(f"\nОшибка при загрузке страницы {page}: {e}")
        time.sleep(5)                               # пауза перед следующей попыткой
        continue

    soup = BeautifulSoup(resp.text, "html.parser")
    books = soup.find_all("article", class_="product_pod")

    page_data = []
    for book in books:
        # Название
        title_tag = book.h3.a
        title = title_tag.get("title", "No title")

        # Цена
        price_tag = book.find("p", class_="price_color")
        price = price_tag.text.strip() if price_tag else "N/A"

        # Наличие
        avail_tag = book.find("p", class_="instock availability")
        availability = avail_tag.text.strip() if avail_tag else "Unknown"

        # Рейтинг
        rating_tag = book.find("p", class_="star-rating")
        rating = get_rating(rating_tag)

        # Ссылка на детальную страницу книги
        relative_url = title_tag.get("href", "")
        full_url = requests.compat.urljoin(BASE_URL, relative_url)

        page_data.append([title, price, rating, availability, full_url])

    # Сохраняем данные со страницы сразу в CSV
    with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerows(page_data)

    # Случайная задержка между запросами
    sleep_time = uniform(1.5, 3.5)
    time.sleep(sleep_time)

print(f"\nПарсинг завершён. Данные сохранены в {OUTPUT_CSV}")

Сбор книг: 100%|██████████| 50/50 [02:52<00:00,  3.45s/it]


Парсинг завершён. Данные сохранены в books.csv
